# Aggregating all player data for in-depth analyses and comparisons

In [1]:
# libraries
import pandas as pd
import numpy as np
import time
import re

#### Getting the standard stats for all players playing for clubs from the Top5 leagues in 2018/19

In [42]:
# creating list of urls for each top5 league club from the 18-19 season
url_list = open('urls/matchday_reports.txt').read().splitlines()
print(url_list[-1])
print(len(url_list))

https://fbref.com/en/matches/96de6624/Tottenham-Hotspur-Liverpool-June-1-2019-Champions-League
51


In [41]:
tables = pd.read_html(url_list[-1], attrs={"id": "shots_822bd0ba"})
df = tables[0]
df

Unnamed: 0_level_0      Unnamed: 1_level_0 Unnamed: 2_level_0  \
               Minute                  Player              Squad   
0                 2.0     Mohamed Salah (pen)          Liverpool   
1                17.0  Trent Alexander-Arnold          Liverpool   
2                21.0           Mohamed Salah          Liverpool   
3                22.0           Mohamed Salah          Liverpool   
4                38.0        Andrew Robertson          Liverpool   
5                39.0           Mohamed Salah          Liverpool   
6                42.0        Jordan Henderson          Liverpool   
7                42.0           Mohamed Salah          Liverpool   
8                 NaN                     NaN                NaN   
9                53.0                 Fabinho          Liverpool   
10               54.0           Mohamed Salah          Liverpool   
11               69.0            James Milner          Liverpool   
12               83.0        Jordan Henderson          Liverpool   
13               87.0         Virgil van Dijk          Liverpool   
14               87.0            Divock Origi          Liverpool   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   xG               PSxG            Outcome   
0                0.79               0.91               Goal   
1                0.02                NaN         Off Target   
2                0.03                NaN            Blocked   
3                0.04                NaN         Off Target   
4                0.03               0.06              Saved   
5                0.03                NaN         Off Target   
6                0.03                NaN            Blocked   
7                0.02                NaN            Blocked   
8                 NaN                NaN                NaN   
9                0.03                NaN            Blocked   
10               0.11                NaN            Blocked   
11               0.05                NaN         Off Target   
12               0.03                NaN         Off Target   
13               0.05                NaN            Blocked   
14               0.04               0.58               Goal   

   Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0  \
             Distance          Body Part              Notes   
0                13.0          Left Foot                NaN   
1                33.0         Right Foot                NaN   
2                27.0          Left Foot                NaN   
3                16.0          Left Foot                NaN   
4                24.0          Left Foot                NaN   
5                23.0         Right Foot             Volley   
6                32.0         Right Foot                NaN   
7                28.0          Left Foot             Volley   
8                 NaN                NaN                NaN   
9                30.0         Right Foot                NaN   
10               12.0          Left Foot                NaN   
11               17.0          Left Foot                NaN   
12               28.0         Right Foot                NaN   
13               15.0         Right Foot             Volley   
14               15.0          Left Foot                NaN   

                     SCA 1                          SCA 2               
                    Player        Event            Player        Event  
0                      NaN       Fouled  Jordan Henderson  Pass (Live)  
1         Jordan Henderson  Pass (Live)           Fabinho  Pass (Live)  
2          Roberto Firmino  Pass (Live)     Mohamed Salah  Pass (Live)  
3   Trent Alexander-Arnold  Pass (Dead)     Mohamed Salah         Shot  
4               Joël Matip  Pass (Live)           Alisson  Pass (Live)  
5         Andrew Robertson         Shot               NaN          NaN  
6                      NaN          NaN               NaN          NaN  
7         Jordan Henderson  Pass (Live)   Rob

In [144]:
from urllib.parse import urlparse
# Initialize an empty list to store dataframes
dfs = []

# Set the delay between requests (in seconds)
delay_between_requests = 5  # Adjust this value as needed

for url in url_list:
    try:
        # Read HTML tables from the URL
        tables = pd.read_html(url, attrs={"id": "shots_822bd0ba"})
        df = tables[0]

        # Append the dataframe to the list
        dfs.append(df)
        
        # Introduce a delay before making the next request
        time.sleep(delay_between_requests)
    except Exception as e:
        print(f"Error reading data from {url}: {str(e)}")

# Concatenate all dataframes into one
total_df = pd.concat(dfs, ignore_index=True)

In [146]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   (Unnamed: 0_level_0, Minute)     766 non-null    object 
 1   (Unnamed: 1_level_0, Player)     766 non-null    object 
 2   (Unnamed: 2_level_0, Squad)      766 non-null    object 
 3   (Unnamed: 3_level_0, xG)         766 non-null    float64
 4   (Unnamed: 4_level_0, PSxG)       275 non-null    float64
 5   (Unnamed: 5_level_0, Outcome)    766 non-null    object 
 6   (Unnamed: 6_level_0, Distance)   766 non-null    float64
 7   (Unnamed: 7_level_0, Body Part)  766 non-null    object 
 8   (Unnamed: 8_level_0, Notes)      117 non-null    object 
 9   (SCA 1, Player)                  729 non-null    object 
 10  (SCA 1, Event)                   733 non-null    object 
 11  (SCA 2, Player)                  621 non-null    object 
 12  (SCA 2, Event)        

In [147]:
total_df.columns

MultiIndex([('Unnamed: 0_level_0',    'Minute'),
            ('Unnamed: 1_level_0',    'Player'),
            ('Unnamed: 2_level_0',     'Squad'),
            ('Unnamed: 3_level_0',        'xG'),
            ('Unnamed: 4_level_0',      'PSxG'),
            ('Unnamed: 5_level_0',   'Outcome'),
            ('Unnamed: 6_level_0',  'Distance'),
            ('Unnamed: 7_level_0', 'Body Part'),
            ('Unnamed: 8_level_0',     'Notes'),
            (             'SCA 1',    'Player'),
            (             'SCA 1',     'Event'),
            (             'SCA 2',    'Player'),
            (             'SCA 2',     'Event')],
           )

In [115]:
total_df

Unnamed: 0_level_0      Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Minute                  Player              Squad   
0                   17              Sadio Mané          Liverpool   
1                   18         Roberto Firmino          Liverpool   
2                   19           Mohamed Salah          Liverpool   
3                   24  Trent Alexander-Arnold          Liverpool   
4                   25     Georginio Wijnaldum          Liverpool   
..                 ...                     ...                ...   
812               54.0           Mohamed Salah          Liverpool   
813               69.0            James Milner          Liverpool   
814               83.0        Jordan Henderson          Liverpool   
815               87.0         Virgil van Dijk          Liverpool   
816               87.0            Divock Origi          Liverpool   

    Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                    xG               PSxG            Outcome   
0                 0.04                NaN            Blocked   
1                 0.59                NaN         Off Target   
2                 0.74               0.95               Goal   
3                 0.06               0.57              Saved   
4                 0.09                NaN         Off Target   
..                 ...                ...                ...   
812               0.11                NaN            Blocked   
813               0.05                NaN         Off Target   
814               0.03                NaN         Off Target   
815               0.05                NaN            Blocked   
816               0.04               0.58               Goal   

    Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0  \
              Distance          Body Part              Notes   
0                 16.0         Right Foot                NaN   
1                  5.0         Right Foot             Volley   
2                  4.0         Right Foot                NaN   
3                 31.0         Right Foot          Free kick   
4                 12.0          Left Foot                NaN   
..                 ...                ...                ...   
812               12.0          Left Foot                NaN   
813               17.0          Left Foot                NaN   
814               28.0         Right Foot                NaN   
815               15.0         Right Foot             Volley   
816               15.0          Left Foot                NaN   

                   SCA 1                                SCA 2               
                  Player        Event                  Player        Event  
0       Andrew Robertson  Pass (Live)     Georginio Wijnaldum  Pass (Live)  
1           James Milner  Pass (Live)  Trent Alexander-Arnold  Pass (Live)  
2       Andrew Robertson  Pass (Live)              Naby Keïta  Pass (Live)  
3        Roberto Firmino       Fouled        Andrew Robertson  Pass (Live)  
4             Naby Keïta  Pass (Live)              Naby Keïta      Take-On  
..                   ...          ...                     ...          ...  
812  Georginio Wijnaldum  Pass (Live)              Joël Matip  Pass (Live)  
813        Mohamed Salah  Pass (Live)              Sadio Mané  Pass (Live)  
814                  NaN          NaN                     NaN          NaN  
815              Alisson  Pass (Live)                     NaN          NaN  
816           Joël Matip  Pass (Live)         Virgil van Dijk         Shot  

[817 rows x 13 columns]

In [187]:
df_clean_total = total_df.copy()

In [188]:
df_clean_total.columns = df_clean_total.columns.get_level_values(1)

In [189]:
df_clean_total.columns

Index(['Minute', 'Player', 'Squad', 'xG', 'PSxG', 'Outcome', 'Distance',
       'Body Part', 'Notes', 'Player', 'Event', 'Player', 'Event'],
      dtype='object')

In [190]:
# Get the last 4 column names
df_clean_total = df_clean_total.iloc[:, :-4]

In [191]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Minute     766 non-null    object 
 1   Player     766 non-null    object 
 2   Squad      766 non-null    object 
 3   xG         766 non-null    float64
 4   PSxG       275 non-null    float64
 5   Outcome    766 non-null    object 
 6   Distance   766 non-null    float64
 7   Body Part  766 non-null    object 
 8   Notes      117 non-null    object 
dtypes: float64(3), object(6)
memory usage: 57.6+ KB


In [181]:
df_clean_total.iloc[:, 1].value_counts()

Player
Mohamed Salah              184
Sadio Mané                 111
Roberto Firmino             91
Georginio Wijnaldum         42
Virgil van Dijk             41
Trent Alexander-Arnold      41
Naby Keïta                  32
James Milner                31
Xherdan Shaqiri             30
Daniel Sturridge            27
Jordan Henderson            25
Andrew Robertson            22
Fabinho                     20
Divock Origi                16
Joël Matip                  14
Adam Lallana                 8
Joe Gomez                    7
Dejan Lovren                 6
James Milner (pen)           5
Mohamed Salah (pen)          5
Nathaniel Clyne              3
Alberto Moreno               2
Sadio Mané (pen)             1
Roberto Firmino (pen)        1
Alex Oxlade-Chamberlain      1
Name: count, dtype: int64

In [182]:
df_clean_total.iloc[:, 0].unique()

array(['17', '18', '19', '24', '25', '36', '37', '45+2', nan, '53', '65',
       '68', '70', '73', '78', '80', '88', '90', '3', '23', '27', '35',
       '44', '45', '49', '60', '61', '77', '81', '84', '87', '90+3', '5',
       '9', '15', '31', '34', '46', '58', '69', '72', '86', '90+2', 4.0,
       10.0, 16.0, 21.0, 43.0, 45.0, 54.0, 82.0, 86.0, '6', '22', '39',
       '43', '54', '63', '90+1', '7', '10', '26', '30', '32', '82', '89',
       '4', '21', '45+1', '45+3', '85', 5.0, 11.0, 15.0, 18.0, 32.0, 44.0,
       58.0, 59.0, 70.0, 72.0, 84.0, 85.0, 89.0, 26.0, 38.0, 68.0, 12.0,
       50.0, 62.0, 65.0, 69.0, 79.0, 17.0, 24.0, 47.0, 49.0, 60.0, 64.0,
       88.0, '11', '20', '38', '48', '51', '56', '76', '90+4', '13', '41',
       '42', '66', '79', 9.0, 13.0, 23.0, 61.0, 74.0, 83.0, '8', '12',
       '33', '71', '74', 20.0, 22.0, 25.0, 28.0, 30.0, 35.0, 41.0, 53.0,
       81.0, 90.0, 39.0, 42.0, 67.0, 76.0, '29', '57', '59', '90+6', '47',
       '52', '62', 14.0, 19.0, 48.0, 77.0, '16

In [204]:
df_clean_total.iloc[:, 0].unique()

array([17., 18., 19., 24., 25., 36., 37., 45., nan, 53., 65., 68., 70.,
       73., 78., 80., 88., 90.,  3., 23., 27., 35., 44., 49., 60., 61.,
       77., 81., 84., 87.,  5.,  9., 15., 31., 34., 46., 58., 69., 72.,
       86.,  4., 10., 16., 21., 43., 54., 82.,  6., 22., 39., 63.,  7.,
       26., 30., 32., 89., 85., 11., 59., 38., 12., 50., 62., 79., 47.,
       64., 20., 48., 51., 56., 76., 13., 41., 42., 66., 74., 83.,  8.,
       33., 71., 28., 67., 29., 57., 52., 14.,  2., 75., 55.,  1.])

In [206]:
df_clean_total["Minute"] = df_clean_total["Minute"].astype("Int64")

In [200]:
def convert_to_int(value):
    if pd.isna(value):
        return value
    if isinstance(value, str):
        return int(value.strip()[:2])
    elif isinstance(value, float):
        return int(value)
    else:
        return None

# Apply the function to the Minutes column
df_clean_total['Minute'] = df_clean_total['Minute'].apply(convert_to_int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Minute     766 non-null    float64
 1   Player     766 non-null    object 
 2   Squad      766 non-null    object 
 3   xG         766 non-null    float64
 4   PSxG       275 non-null    float64
 5   Outcome    766 non-null    object 
 6   Distance   766 non-null    float64
 7   Body Part  766 non-null    object 
 8   Notes      117 non-null    object 
dtypes: float64(4), object(5)
memory usage: 57.6+ KB


In [207]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Minute     766 non-null    Int64  
 1   Player     766 non-null    object 
 2   Squad      766 non-null    object 
 3   xG         766 non-null    float64
 4   PSxG       275 non-null    float64
 5   Outcome    766 non-null    object 
 6   Distance   766 non-null    float64
 7   Body Part  766 non-null    object 
 8   Notes      117 non-null    object 
dtypes: Int64(1), float64(3), object(5)
memory usage: 58.4+ KB


In [208]:
df_clean_total.head()

,Minute,Player,Squad,xG,PSxG,Outcome,Distance,Body Part,Notes
0,17,Sadio Mané,Liverpool,0.04,NaN,Blocked,16.0,Right Foot,NaN
1,18,Roberto Firmino,Liverpool,0.59,NaN,Off Target,5.0,Right Foot,Volley
2,19,Mohamed Salah,Liverpool,0.74,0.95,Goal,4.0,Right Foot,NaN
3,24,Trent Alexander-Arnold,Liverpool,0.06,0.57,Saved,31.0,Right Foot,Free kick
4,25,Georginio Wijnaldum,Liverpool,0.09,NaN,Off Target,12.0,Left Foot,NaN


In [210]:
# values for 90 and 45 include shots attempted in extra time (found no better solution)
df_clean_total["Minute"].value_counts()

Minute
90    44
45    20
87    17
82    16
81    15
      ..
8      3
5      3
46     3
21     3
1      2
Name: count, Length: 89, dtype: Int64

In [173]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Minute     486 non-null    float64
 1   Player     766 non-null    object 
 2   Squad      766 non-null    object 
 3   xG         766 non-null    float64
 4   PSxG       275 non-null    float64
 5   Outcome    766 non-null    object 
 6   Distance   766 non-null    float64
 7   Body Part  766 non-null    object 
 8   Notes      117 non-null    object 
dtypes: float64(4), object(5)
memory usage: 57.6+ KB


In [211]:
df_clean_total.to_csv("all_shots_CLandPL.csv", index=False)